In [93]:
import numpy as np
import simpy
from math import ceil

In [109]:
Ln = 10
L1 = 20
L2 = 10
Lr = 30
L3 = 10

Mu3 = 1.5
DeltaMu3 = 0.4
MuV = 5
DeltaMuV = 2
Mu1 = 5
DeltaMu1 = 1
Mu2 = 25
DeltaMu2 = 0.5

Tau = 24

time_unit = 0.001
memory = 0 
task_id = 0

created = []
translated = []
edited = []

created_cnt = 0
translated_cnt = 0
edited_cnt = 0
processed_cnt = 0

cur_translated = -1
cur_edited = -1
cur_processed = -1


def init():
    global memory, task_id, created_cnt, translated_cnt, edited_cnt, processed_cnt, cur_translated, cur_edited, cur_processed
    memory = 0 
    task_id = 0
    
    created = []
    translated = []
    edited = []

    created_cnt = 0
    translated_cnt = 0
    edited_cnt = 0
    processed_cnt = 0
    
    cur_translated = -1
    cur_edited = -1
    cur_processed = -1
    
def get_rand(start, offset):
    return start + np.random.uniform(-offset,offset)
    
def task_creator(env, proc):
    global memory, task_id, created_cnt
    while True:
        with proc.request() as req:
            yield req
            timeout = get_rand(MuV, DeltaMuV)
            print('%d task started to be created at \t %f \t %d \t Created: %s' % (task_id, env.now, memory, str(created)))
            yield env.timeout(timeout)
            memory += Ln
            created.append(task_id)
            task_id += 1
            created_cnt += 1
            print('%d task finished to be created at \t %f \t %d \t Created: %s' % (task_id - 1, env.now, memory,
                                                                                    str(created)))
            

def first_processor(env, proc):
    global memory, translated_cnt, cur_translated
    while True:
        with proc.request() as req:
            if(len(created) > 0):
                yield req
                timeout = get_rand(Mu1, DeltaMu1)
                cur_translated = created.pop(0)
                print('%d task started to be translated at \t %f \t %d \t Translated: %s' % (cur_translated, env.now, 
                                                                                             memory, str(translated)))
                yield env.timeout(timeout)
                memory += L2 - Ln
                translated.append(cur_translated)
                translated_cnt += 1    
                print('%d task finished to be translated at \t %f \t %d \t Translated: %s' % (cur_translated, env.now,
                                                                                              memory, str(translated)))
                cur_translated = -1
            else:
                yield env.timeout(time_unit)


def second_processor(env, proc):
    global memory, edited_cnt, cur_edited
    while True:
        with proc.request() as req:
            if(len(translated) > 0):
                yield req       
                #DONT NEED TIMEOUT FOR EACH L2 BECAUSE TRANSLATED SIZE = L2
                timeout = get_rand(Mu2, DeltaMu2)
                cur_edited = translated.pop(0)
                print('%d task started to be edit at \t \t %f \t %d \t Edited: %s' % (cur_edited, env.now, memory, 
                                                                                      str(edited)))              
                yield env.timeout(timeout)
                memory += Lr - L2
                edited.append(cur_edited)
                edited_cnt += 1  
                print('%d task finished to be edit at \t \t %f \t %d \t Edited: %s' % (cur_edited, env.now, memory, 
                                                                                       str(edited)))
                cur_edited = -1
            else:
                yield env.timeout(time_unit)


def third_processor(env, proc):
    global memory, processed_cnt, cur_processed
    while True:
        with proc.request() as req:
            if(len(edited) > 0):
                yield req
                cur_processed = edited.pop(0)
                print('%d task started to be processed at \t %f \t %d' % (cur_processed, env.now, memory))
                for i in range(ceil(Lr / L3)):
                    print('%d task processed. %d iteration at \t %f' % (cur_processed, i, env.now))
                    timeout = get_rand(Mu3, DeltaMu3)
                    yield env.timeout(timeout)
                memory -= Lr
                processed_cnt += 1
                print('%d task finished to be processed at \t %f \t %d' % (cur_processed, env.now, memory))
                cur_processed = -1
            else:
                yield env.timeout(time_unit)

In [110]:
init()
env = simpy.Environment()
creator_processor_resource = simpy.Resource(env, capacity=1)
first_processor_resource = simpy.Resource(env, capacity=1)
second_processor_resource = simpy.Resource(env, capacity=1)
third_processor_resource = simpy.Resource(env, capacity=1)

env.process(task_creator(env, creator_processor_resource))
env.process(first_processor(env, first_processor_resource))
env.process(second_processor(env, second_processor_resource))
env.process(third_processor(env, third_processor_resource))

print("Start Modeling")
env.run(until=60)
print("Finish Modeling")

Start Modeling
0 task started to be created at 	 0.000000 	 0 	 Created: []
0 task finished to be created at 	 4.205821 	 10 	 Created: [0]
1 task started to be created at 	 4.205821 	 10 	 Created: [0]
0 task started to be translated at 	 4.206000 	 10 	 Translated: []
1 task finished to be created at 	 9.591897 	 20 	 Created: [1]
2 task started to be created at 	 9.591897 	 20 	 Created: [1]
0 task finished to be translated at 	 10.150390 	 20 	 Translated: [0]
1 task started to be translated at 	 10.150390 	 20 	 Translated: [0]
0 task started to be edit at 	 	 10.151000 	 20 	 Edited: []
2 task finished to be created at 	 14.273026 	 30 	 Created: [2]
3 task started to be created at 	 14.273026 	 30 	 Created: [2]
1 task finished to be translated at 	 15.571276 	 30 	 Translated: [1]
2 task started to be translated at 	 15.571276 	 30 	 Translated: [1]
3 task finished to be created at 	 19.614582 	 40 	 Created: [3]
4 task started to be created at 	 19.614582 	 40 	 Created: [3]
2

In [111]:
print("Report:")
print("Memory State: %d" % memory)

print("For Translation: " + str(created) + " Current: " + str(cur_translated))
print("For Editing: " + str(translated) + " Edited: " + str(cur_edited))
print("For Processing: " + str(edited) + " Processed: " + str(cur_processed))

print("All Time Created: %d" % created_cnt)
print("All Time Translated: %d" % translated_cnt)
print("All Time Edited: %d" % edited_cnt)
print("All Time Processed: %d" % processed_cnt)

Report:
Memory State: 100
For Translation: [] Current: -1
For Editing: [2, 3, 4, 5, 6, 7, 8, 9, 10] Edited: 1
For Processing: [] Processed: -1
All Time Created: 11
All Time Translated: 11
All Time Edited: 1
All Time Processed: 1
